In [ ]:
import pandas as pd
from pathlib import Path
import glob
import sqlite3

In [ ]:
conn = sqlite3.connect("mimic_iv.db")

In [ ]:
existing_tables = pd.read_sql_query(
    "SELECT name FROM sqlite_master WHERE type='table';", conn
)['name'].tolist()

folder_hosp = Path("files/mimiciv/3.1/hosp")

for file in folder_hosp.glob("*.csv.gz"):
    table_name = file.stem
    if table_name in existing_tables:
        print(f"Skipping {table_name}, already in database.")
        continue

    try:
        print(f"Inserting {table_name} in chunks...")
        chunks = pd.read_csv(file, compression='gzip', chunksize=50000, low_memory=False)

        for i, chunk in enumerate(chunks):
            chunk.to_sql(table_name, conn, if_exists='append', index=False)
            print(f"Inserted chunk {i + 1}")

    except Exception as e:
        print(f"Failed on {file.name}: {e}")
